<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>

# Laboratory work #5. FHIR resource Encounter

John Thompson finally arrives at the clinical center and has just started his visit with the practitioner Kelly Smith.
He told Kelly about his concerns about fever and high blood pressure.
So, Kelly checked the current temperature and the blood pressure: 96.8 F and 130/80.
Kelly diagnosed hypertension and requested some blood tests to diagnose the cause of the fever.

Let’s create an Encounter to track the visit.

## 1 Introduction

### 1.1 Import library

For the beginning we should import the `fhirpy` and `os` libraries.  
Also we import `pprint` helper function from local `utils.py` file. We'll use `pprint` to display some structures.

In [ ]:
import os
import fhirpy

from utils import pprint

### 1.2 Create an instance of connection

To load data from FHIR server we should instantiate `FHIRClient` class from `fhirpy` package.  
We pass `url` and `authorization` arguments from environment.

In [ ]:
client = fhirpy.FHIRClient(
    url=os.environ['BOX_URL'] + '/fhir', 
    authorization=os.environ['BOX_AUTHORIZATION'])

Now, we are able to operate with FHIR resources using `client`.

## 2 Load related resources

**Exercise**: Before creating `Encounter` resource you need to find the patient - `John Thompson`

In [ ]:
patient = client.resources('Patient').search(
    # Write code here
).first()
patient

**Exercise**: Next you need to find the practitioner - `Kelly Smith`

In [ ]:
practitioner = client.resources('Practitioner').search(
    # Write code here
).first()
practitioner

The clinical center where our patient has finally arrived is `Hahnemann University Hospital`

**Exercise**: Find `Location` resource for the clinical center 'Hahnemann University Hospital'

In [ ]:
location = client.resources('Location').search(
    # Write code here
).first()
location

And finally load `Appointment` that scheduled this `Encounter`

In [ ]:
appointment = client.resources('Appointment').search(actor=patient['id']).first()
appointment

## 3 Encounter Resource

[Encounter](https://www.hl7.org/fhir/encounter.html) is an interaction between a patient and healthcare provider for the purpose of providing healthcare service or assessing the health status of a patient.

An `Encounter` encompasses the lifecycle from pre-admission, the actual encounter, admission, stay and discharge. During the encounter the patient may move from practitioner to practitioner and location to location.

`Encounter` has a broad scope, so some components may be irrelevant to a specific context, the `class` element is used to distinguish between settings.

Different orgranizations may have different level of aggregation for `Encounter`. For example, during hospitalization each single visit of a practitioner may lead to a new instance of `Encounter`, but in other orgranization the whole hospitalization may be defined in terms of a single `Encounter`.  

We use `Appointment` for the planning stage. When the appointment is about to start, we create a new `Encounter` and mark `Appointment` object as fulfilled.

### 3.1 Load Encounter resources

Let's search for existent `Encounter` resources before creating one

In [ ]:
encounters = client.resources('Encounter').search(subject=patient['id']).first()
encounters

### 3.2 Create Condition

[Condition](https://www.hl7.org/fhir/condition.html) resource can be used to describe information about illness identified from application of clinical reasoning. It is important to distinguish `Condition` from `Observaion`, we use `Observation` when a symptom contributes to the establishment of a condition. `Condition` on the other hand is about a clinician's assessment and assertion of a particular aspect of a patient's state of healt.

Let's create `Condition` for hypertension. Hypertension is encoded as `38341003` in `http://snomed.info/sct` system.

In [ ]:
condition = client.resource(
    'Condition',
    id='condition_for_john_thompson',
    code={'coding': [{'system': 'http://snomed.info/sct', 'code': '38341003'}]},
    subject=patient
)
condition

### 3.3 Create Encounter

Now that we've all the information required, we can finally register our `Encounter`

In [ ]:
encounter = client.resource(
    'Encounter',
    id='encounter_for_john_thompson',
    status='in-progress',
    subject=patient,
    participant=[{
        'type': [{
            'coding': [{
                'system': 'http://hl7.org/fhir/v3/ParticipationType',
                'code': 'PPRF'
            }]
        }],
        'individual': practitioner
    }],
    appointment=appointment,
    diagnosis=[{'condition': condition}],
    location=[{'location': location, 'status': 'active'}]
)
encounter

### 3.4 Register symptoms with Observation

We can link `Observation` resource to `Encounter` with `Observation.context`.

**Exercise**: You need to register the observations:
* temperature 96.8 F
* blood pressure 130/80

and link them to the encounter we've just created.

In [ ]:
temperature_observation = client.resource(
    'Observation',
    id='temperature_observation_for_john_thompson',
    status='preliminary',
    category=[{
        'coding': [{
            'system': 'http://hl7.org/fhir/observation-category',
            'code': 'vital-signs'
        }]
    }],
    code={
        'coding': [{
            'system': 'http://loinc.org',
            'code': '8310-5'
        }]
    },
    effectiveDateTime='2018-09-16',
    # Write code here
)
temperature_observation

In [ ]:
blood_pressure_observation = client.resource(
    'Observation',
    id='blood_pressure_observation_for_john_thompson',
    status='preliminary',
    category=[{
        'coding': [{
            'system': 'http://hl7.org/fhir/observation-category',
            'code': 'vital-signs'
        }]
    }],
    code={
        'coding': [{
            'system': 'http://loinc.org',
            'code': '55284-4'
        }]
    },
    effectiveDateTime='2018-09-16',
    # Write code here
)
blood_pressure_observation

## Summary

In this laboratory work we've discussed:
* How to search for `Encounter`
* How to create `Condition`
* What is the difference between `Condition` and `Observation`
* How to create `Encounter`
* How to link `Condition` and `Observation` to `Encounter`

<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>